In [8]:
import json
import urllib.parse
import requests
import csv
import sys
import urllib.request
import os
import glob
import pandas
import datetime

##
# Convert to string keeping encoding in mind...
##
def to_string(s):
    try:
        return str(s)
    except:
        #Change the encoding type if needed
        return s.encode('utf-8')

def validate(date_text):
    try:
        datetime.datetime.strptime(date_text, '%Y,%m,%d')
    except ValueError:
        raise ValueError("Incorrect data format, should be DD,MM,YYYY")
    
def reduce_item(key, value):
    global reduced_item
    
    #Reduction Condition 1
    if type(value) is list:
        i=0
        for sub_item in value:
            reduce_item(key+'_'+to_string(i), sub_item)
            i=i+1

    #Reduction Condition 2
    elif type(value) is dict:
        sub_keys = value.keys()
        for sub_key in sub_keys:
            reduce_item(key+'_'+to_string(sub_key), value[sub_key])
    
    #Base Condition
    else:
        reduced_item[to_string(key)] = to_string(value)

if __name__ == "__main__":
        #Reading arguments
        node = "price_usd"
        time_text = datetime.datetime.strptime("00:00:00", "%H:%M:%S")
        epoch = datetime.datetime.utcfromtimestamp(0)
        csv_file_path = r'Type the path here.csv'
        
        main_api = "https://graphs2.coinmarketcap.com/currencies/"
        crypto_name = input ("Type the crypto name as writen on CMM URL") + '/'
        Ti = input ('Type initial date in the format:  YYYY,MM,DD')
        validate (Ti)
        Ti = datetime.datetime.strptime(Ti, "%Y,%m,%d")
        Ti = datetime.datetime.combine(Ti.date(), time_text.time())
        Tf = input ('Type end date in the format: YYYY,MM,DD')
        validate (Tf)
        Tf = datetime.datetime.strptime(Tf, "%Y,%m,%d")
        Tf = datetime.datetime.combine(Tf.date(), time_text.time())
        days = (Tf - Ti).days
        timestamp1 = Ti - epoch
        timestamp1 = str(int(timestamp1.total_seconds()))

        
        i=0
        datapros= []
        for i in range(days):
            ti = int(timestamp1) ++ (86400*i)
            tf = ti ++ (86400)
            url1 = main_api + crypto_name + str(ti) +'000/'+ str(tf) +'000/'
            i = i+1
            #print (url1)
            with urllib.request.urlopen(url1) as url:
                data = json.loads(url.read().decode())

            try:
                data_to_be_processed = data[node]
            except:
                data_to_be_processed = data
            datapros= datapros+data_to_be_processed
        processed_data = []
        header = []

        for item in datapros:
            reduced_item = {}
            reduce_item(node, item)

            header += reduced_item.keys()
            processed_data.append(reduced_item)
        header = list(set(header))
        header.sort()
        with open(csv_file_path, 'w+') as f:
            writer = csv.DictWriter(f,header)
            for row in processed_data:
                writer.writerow(row)

Qual moeda?eos
Digite a data inicial YYYY,MM,DD2017,07,01
Digite a data final YYYY,MM,DD2018,06,20
